# Image segmentation using U-Nets (Data Science Bowl 2018)

### Segmentation of nuclei in microscopy images
The segmentation of nuclei in microscopy images is a common task in many workflows. Most traditional image processing methods are created for one specific type of image, and perform poorly on others.

The aim is to develop a general-purpose algorithm to segment cell nuclei from any type of microscopy images.

<img src="assets/nuclei_image_types.png" style="width: 600px;"/>

---

We will use a U-Net Convolutional Neural Network to perform this task. This notebook is adapted from [Keras U-Net starter by Kjetil Åmdal-Sævik](https://www.kaggle.com/keegil/keras-u-net-starter-lb-0-277?scriptVersionId=2164855) ([Licensed under the Apache License, Version 2.0](http://www.apache.org/licenses/LICENSE-2.0))


**For more information, see the original publication about the 2018 Data Science Bowl:**

[Caicedo, J.C., Goodman, A., Karhohs, K.W. et al.
Nucleus segmentation across imaging experiments: the 2018 Data Science Bowl.
Nat Methods 16, 1247–1253 (2019)](https://doi.org/10.1038/s41592-019-0612-7)

We will be using some data from the original challenge (hosted at [https://www.kaggle.com/c/data-science-bowl-2018](https://www.kaggle.com/c/data-science-bowl-2018) (account required), and also from the [Broad Institute](https://bbbc.broadinstitute.org/BBBC038)).

### **A copy of the data for this course is available [here](https://drive.google.com/file/d/1hlDAvRFH7Ax_Nxee4jCyN_IIDax-CLc8/view?usp=sharing)**

## We begin by loading and visualising some of the data

## Load the data
We're using [pooch](https://www.fatiando.org/pooch/) to automate downloading the data, and check the hash.

**Run the following cell to download the data in advance**

In [ ]:
import pooch
from pathlib import Path
data_path = Path.cwd().parent / "dsb_data"

# Use pooch to fetch data if it hasn't already been downloaded
dsb_data_url = "https://gin.g-node.org/neuroinformatics/image-analysis-courses/raw/master/dsb_2018/stage1_train.zip"
data_path = pooch.retrieve(dsb_data_url, known_hash="4e9fb804d2ff054d7bd83d4781c66e07dae9937207a09168330a5b5cefa0daba", progressbar=True, processor=pooch.Unzip(extract_dir=data_path))
data_path = Path(data_path[0]).parent.parent.parent

In [ ]:
import os
import numpy as np

from tqdm import tqdm
from skimage.io import imread
from skimage.transform import resize

DEBUG = True  # Set to True to only load a fraction of the raw data


def load_images(
    data_path, image_width=128, image_height=128, num_channels=3, debug=False
):
    """
    :param data_path: Location of the data on disk (pathlib object)
    :param filenames: List of image paths
    :param image_width: Image width in pixels for resizing
    :param image_height: Image height in pixels for resizing
    :param num_channels: Final number of channels required

    :return images, masks: Numpy array of images, and segmentation masks
    """

    data_path = Path(data_path)

    filenames = next(os.walk(data_path))[1]
    if debug:
        filenames = filenames[:40]
        
    images = np.zeros(
        (len(filenames), image_height, image_width, num_channels),
        dtype=np.uint8,
    )
    masks = np.zeros((len(filenames), image_height, image_width, 1), dtype=bool)

    for n, image_id in tqdm(enumerate(filenames), total=len(filenames)):
        img = imread(data_path / image_id / "images" / (image_id + ".png"))[
            :, :, :num_channels
        ]
        img = resize(
            img,
            (image_height, image_width),
            mode="constant",
            preserve_range=True,
        )
        images[n] = img
        mask = np.zeros((image_height, image_width, 1), dtype=bool)

        mask_dir = data_path / image_id / "masks"
        for mask_file in next(os.walk(mask_dir))[2]:
            mask_ = imread(mask_dir / mask_file)
            mask_ = np.expand_dims(
                resize(
                    mask_,
                    (image_height, image_width),
                    mode="constant",
                    preserve_range=True,
                ),
                axis=-1,
            )
            mask = np.maximum(mask, mask_)
        masks[n] = mask
    return images, masks

In [ ]:
images, masks = load_images(data_path, debug=DEBUG)

In [ ]:
# Visualise some raw data and segmentation masks
import random

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid


def plot_raw_data(raw_images, masks, num_to_plot=2, fig_size=(10, 10), axes_pad=0.1):
    """
    Plot raw data

    :param raw_images: Array of raw data images
    :param masks: Array of mask images
    :param num_to_plot: How many random images to plot
    :param fig_size: Tuple of overall figure size
    :param axes_pad: Seperation of individual images
    """

    idx_to_plot = tuple(random.sample(range(0, len(raw_images)), num_to_plot))

    raw_images = raw_images[tuple([idx_to_plot])]
    masks = np.squeeze(masks)[tuple([idx_to_plot])]

    fig = plt.figure(figsize=fig_size)
    grid = ImageGrid(
        fig,
        111,
        nrows_ncols=(num_to_plot, 2),
        axes_pad=axes_pad,
    )

    images = []
    for a, b in zip(raw_images, masks):
        images.extend([a, b])

    for ax, im in zip(grid, images):
        ax.imshow(im)
        ax.axis("off")

    grid[0].title.set_text("Raw data")
    grid[1].title.set_text("Segmentation masks")


plot_raw_data(images, masks, num_to_plot=5)

## Now we'll build a model to segment these images

We'll be using the popular [U-Net architecture](https://arxiv.org/abs/1505.04597)

<img src="assets/unet.png" style="width: 400px;"/>

In [1]:
# This will import TensorFlow, which will likely print lots of warnings to the user.
# These can generally be safely ignored. 

from keras.models import Model
from keras.layers import Input
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers import concatenate


def build_unet(image_width=128, image_height=128, num_channels=3):
    inputs = Input((image_height, image_width, num_channels))
    s = Lambda(lambda x: x / 255)(inputs)

    c1 = Conv2D(8, (3, 3), activation="relu", padding="same")(inputs)
    c1 = Conv2D(8, (3, 3), activation="relu", padding="same")(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(16, (3, 3), activation="relu", padding="same")(p1)
    c2 = Conv2D(16, (3, 3), activation="relu", padding="same")(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(32, (3, 3), activation="relu", padding="same")(p2)
    c3 = Conv2D(32, (3, 3), activation="relu", padding="same")(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(64, (3, 3), activation="relu", padding="same")(p3)
    c4 = Conv2D(64, (3, 3), activation="relu", padding="same")(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(128, (3, 3), activation="relu", padding="same")(p4)
    c5 = Conv2D(128, (3, 3), activation="relu", padding="same")(c5)

    u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding="same")(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(64, (3, 3), activation="relu", padding="same")(u6)
    c6 = Conv2D(64, (3, 3), activation="relu", padding="same")(c6)

    u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding="same")(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(32, (3, 3), activation="relu", padding="same")(u7)
    c7 = Conv2D(32, (3, 3), activation="relu", padding="same")(c7)

    u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding="same")(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(16, (3, 3), activation="relu", padding="same")(u8)
    c8 = Conv2D(16, (3, 3), activation="relu", padding="same")(c8)

    u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding="same")(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(8, (3, 3), activation="relu", padding="same")(u9)
    c9 = Conv2D(8, (3, 3), activation="relu", padding="same")(c9)

    outputs = Conv2D(1, (1, 1), activation="sigmoid")(c9)
    model = Model(inputs=[inputs], outputs=[outputs])
    return model

2023-06-01 15:41:16.133382: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 15:41:16.241584: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-01 15:41:16.764622: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-01 15:41:16.764651: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

**N.B. For most applications, you wouldn't need to build your own network architecture. Most common archictures are available in open-source packages, including [in keras itself](https://keras.io/api/applications/)**.

In [ ]:
import tensorflow as tf

model = build_unet()
model.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=[
        tf.keras.metrics.MeanIoU(num_classes=2),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.Precision(),
    ],
)

## Split the dataset into a train (80%) and test (20%) fraction

In [ ]:
from sklearn.model_selection import train_test_split

test_fraction = 0.2

(
    images_train,
    images_test,
    masks_train,
    masks_test,
) = train_test_split(
    images,
    masks,
    test_size=test_fraction,
)

## Train the model

In [ ]:
epochs = 100  # Number of iterations over the full dataset
batch_size = 32  # Number of samples per gradient update
validation_split = 0.1  # In addition to test dataset, use 10% of data to assess performance during training

model.fit(
    images_train,
    masks_train,
    validation_split=validation_split,
    batch_size=batch_size,
    epochs=epochs,
)

## Apply network to (unseen) test data

In [ ]:
predicted_masks = model.predict(images_test, verbose=1)
predicted_masks = predicted_masks > 0.5

In [ ]:
# Visualise some results
def plot_predictions(
    raw_images,
    predictions,
    ground_truth,
    num_to_plot=2,
    fig_size=(15, 15),
    axes_pad=0.1,
):
    """
    Compare predicted & actual mask images

    :param raw_images: Array of raw data images
    :param predictions: Array of prediction mask images
    :param ground_truth: Array of ground truth mask images
    :param num_to_plot: How many random images to plot
    :param fig_size: Tuple of overall figure size
    :param axes_pad: Seperation of individual images
    """

    idx_to_plot = tuple(random.sample(range(0, len(raw_images)), num_to_plot))

    raw_images = raw_images[tuple([idx_to_plot])]
    predictions = np.squeeze(predictions)[tuple([idx_to_plot])]
    ground_truth = np.squeeze(ground_truth)[tuple([idx_to_plot])]

    fig = plt.figure(figsize=fig_size)
    grid = ImageGrid(
        fig,
        111,
        nrows_ncols=(num_to_plot, 3),
        axes_pad=axes_pad,
    )

    images = []
    for a, b, c in zip(raw_images, predictions, ground_truth):
        images.extend([a, b, c])

    for ax, im in zip(grid, images):
        ax.imshow(im)
        ax.axis("off")

    grid[0].title.set_text("Raw data")
    grid[1].title.set_text("Network prediction")
    grid[2].title.set_text("Ground truth")

In [ ]:
plot_predictions(images_test, predicted_masks, masks_test, num_to_plot=8)

## Limitations
* Segmentation not perfect, lots of ways to improve (more training, augmentation, better network, adjust hyperparameters etc.)
* We segmented *all* cells, and not *each* cell. We could further these masks (e.g. by using a watershed algorithm), or use an alternative, object detection network (e.g. [Mask R-CNN](https://arxiv.org/abs/1703.06870))